# Prelab 4

In [4]:
# Imports and setup
from pint import UnitRegistry
import math
import numpy as np

# Use pint
units = UnitRegistry()
units.default_format = "~P"

# Define Tesla
ureg = UnitRegistry()
Q_ = ureg.Quantity
ureg.define('Tesla = units.kilogram*units.second**(-2)*units.amp**(-1) = T')


In [17]:
# Known values
L_goal = 20*10**(-6) * units.henry
B_max = 0.2 * units.tesla
D_max = 0.5 # Duty Cycle Max
I_pk = 10 * units.amp
I_rms_cu = 10 * units.amp / (units.mm)**2 # rms

Lg_inches = np.array([
    0.002,
    0.003,
    0.005,
    0.01,
    0.015
]) # inches

Lg = Lg_inches*25.4*units.mm

wires = {
    0.82: 18,
    0.52: 20,
    0.33: 22,
    0.20: 24 
} # mm^2: awg

u_o = 1.2566 * 10**(-6) # free space (we only need this value)

In [9]:
# Current calculation and wire sizing
I_rms = math.sqrt(D_max/3) * I_pk
min_wire_area = I_rms/I_rms_cu
wire_gauge=0
for area in wires:
    if area > min_wire_area:
        wire_gauge = wires[area]
        
print("Min wire gauge:", wire_gauge)

Min wire gauge: 20


In [10]:
# 2 * outer area + inner area
area_finder = lambda t: 2 * ((2*(7+t)*(10.65+t) - (7+t)*math.sqrt((9-t)**2 - (7+t)**2) - (9 - t)**2 * math.asin((7+t)/(9-t)))) + (np.pi*(8.8/2 + t)**2)
reluctance_finder = lambda t: u_o*10**(-3)*area_finder(t)/t
vfunc = np.vectorize(reluctance_finder)
R = vfunc(Lg)

N = np.sqrt(np.divide(L_goal,R))

print("R:", R)

B = 11*I_rms*u_o

print("N:", N)
# print("I_rms", I_rms, ", B:", B)

# print(str(10**2/(20*10**(-6))))

R: [3.48725767e-06 2.37084584e-06 1.47837252e-06 8.10978773e-07
 5.90461972e-07]
N: [2.3948205628454513 2.904446227385534 3.6780959784860303 4.966040470748853 5.819947025588554] H⁰⋅⁵
